## 1) Сбор данных Google Trends

In [24]:
from pytrends.request import TrendReq
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
import csv
# скачаем данные с google doc файла
data = pd.read_csv('https://docs.google.com/spreadsheets/d/10WgVF3KSGS969coU-YMMjY2b_NIUEAPz0jFU1OApwPs/export?format=csv&gid=0')
data=data.loc[data['trends'] == 1]

# возьмём значения по ключевым словам-запросам
kw_list = data['Запрос'].values.tolist()
#kw_list = ['потеря обоняния', 'потеря обоняния при ковид' ]
pytrends = TrendReq(hl='RU', tz=180)

In [83]:
# Параметры google

geo = 'RU'
timeframe = '2020-01-20 2021-04-25'

step=1

In [84]:
# Функция вычисления тренда 

def trends (kw_list):
    pytrends.build_payload(kw_list, timeframe=timeframe, geo=geo, gprop='')
    iot_df = pytrends.interest_over_time()
    iot_df = iot_df.reset_index()
    iot_df = iot_df.drop(['isPartial'], axis=1)
    return iot_df

# Функция сборки всего датасета (step - максимально разрешённое количество одновременных запросов, здесь - 5)

def step_data(step):
    query_df = trends(kw_list[:1])[['date']]
    p=step
    for i in range(0, len(kw_list), p):
        df = trends(kw_list[i:i+p])
        query_df = pd.concat([query_df, df.drop(['date'],axis=1)], axis=1)    
    return query_df

In [87]:
df_api = step_data(step)
df_api.to_excel('raw_data//google.xlsx',index=False)
df_api

,date,потеря обоняния,симптомы коронавируса,Анализы на коронавирус,Признаки коронавируса,КТ легких,Купить Гидроксихлорохин,Как защититься от коронавируса,КОВИД-19,потеря вкуса,...,тромбоз,ЭпиВакКорона,КовиВак,инкубационный период,вирулентность,мутации,Британский штамм,маска,перчатки,профилактика
0,2020-01-26,1,23,9,23,3,0,16,0,0,...,63,0,0,64,41,68,0,24,27,9
1,2020-02-02,2,11,0,10,2,0,8,0,4,...,43,0,0,38,0,52,0,24,26,10
2,2020-02-09,1,5,5,5,1,0,0,0,0,...,53,0,0,30,0,28,0,24,30,8
3,2020-02-16,2,4,0,3,3,0,8,0,4,...,46,0,0,34,10,68,0,20,27,8
4,2020-02-23,0,11,0,12,3,0,8,0,0,...,43,0,0,36,31,51,0,23,23,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2021-03-28,7,4,11,5,10,0,0,25,5,...,54,75,100,22,0,44,7,42,26,7
62,2021-04-04,4,3,0,3,11,0,0,49,10,...,76,100,46,19,12,35,22,39,31,9
63,2021-04-11,10,3,11,4,11,0,0,12,5,...,60,88,62,19,0,24,6,37,30,9
64,2021-04-18,4,3,10,4,12,4,0,12,0,...,100,74,78,20,11,31,7,40,31,8


## 2) Сбор данных по просмотрам страниц WIKI


In [ ]:
!pip install git+https://github.com/Commonists/pageview-api.git

In [4]:
import pageviewapi
from attrdict import AttrDict
import pandas as pd
import datetime

In [10]:
def wikipageview_to_df(n_wikipedia, page_name_list, start_date, stop_date):
  """
  Функция собирает статистику просмотра страницы Википедии в датасет.
  Для работы функции необходимо установвить pageview-API c https://pypi.org/project/pageviewapi/ и attrdict

  Arguments:
    n_wikipedia: версия Википедии в формате str ('ru.wikipedia', 'de.wikipedia')
    page_name_list: перечень страниц 
    start_date: начальная дата в формате 'yyyymmdd'
    stop_date:  конечная дата в формате 'yyyymmdd'
  
  Returns:
        Pandas DataFrame

    """

  import pageviewapi
  from attrdict import AttrDict
  import pandas as pd
  import datetime

  total_wiki_dict = dict()
  

  for i in range(len(page_name_list)-1):
    
    for page_name in page_name_list:

      pageview_dict = pageviewapi.per_article(n_wikipedia, page_name, start_date, stop_date,
                        access='all-access', agent='all-agents', granularity='daily')
    
      timestamp = list()
      views = list()
      article = set()
      timeview_dict = dict()
      article_dict = dict()
      
      for j in range(len(pageview_dict('items'))):
        timestamp.append(pageview_dict('items')[j]('timestamp'))
        views.append(pageview_dict('items')[j]('views'))
        article.add(pageview_dict('items')[j]('article'))
        j+=1
           
      for l,m in zip(timestamp, views):
        timeview_dict[l]=m
      for article in article:
        article_dict[article] = timeview_dict 
    
      total_wiki_dict.update(article_dict)
      i+=1
      wiki_df = pd.DataFrame.from_dict(total_wiki_dict)
      wiki_df['Date'] = wiki_df.index[:]
      wiki_df['Date'] = wiki_df['Date'].map(lambda x: str(x)[:-2])
      wiki_df['Date'] = pd.to_datetime(wiki_df['Date'], format='%Y-%m-%d')
      wiki_df['WD'] = wiki_df['Date'].dt.weekday
      wiki_df = wiki_df.groupby(pd.Grouper(key='Date', freq='W')).sum()
      wiki_df = wiki_df.drop(['WD'], axis = 1)
      
  return wiki_df

# Формируем список запросов и собираем DataFrame

In [88]:
page_name_list = list(['Аносмия',
                       'Пневмония',
                       'Симптом_«матового_стекла»',
                       'COVID-19',
                       'Тестирование_на_коронавирусную_инфекцию_COVID-19',
                       'КТ_грудной_клетки',
                       'Профилактика_COVID-19',
                       'Ремдесивир',
                       'Фавипиравир',
                       'Гидроксихлорохин',
                       'Амоксициллин',
                       'Азитромицин',
                       'Левофлоксацин',
                       'Моксифлоксацин',
                       'Кашель',
                       'Гипербарическая_оксигенация',
                       'Дыхательная_недостаточность',
                       'Острая_дыхательная_недостаточность',
                       'Пульсоксиметрия',
                       'Пульсоксиметр',
                       'Ринит',
                       'Заложенность_носа',
                       'SARS-CoV-2',
                       'Новый_коронавирус',
                       'Жизнеспособность_и_перенос_SARS-CoV-2_вне_организма',
                       'Вакцина_против_COVID-19',
                       'ЭпиВакКорона',
                       'CoronaVac',
                       'КовиВак',
                       'Спутник_V'])
start_date = '20190901'
stop_date = '20210424'

In [89]:
df = wikipageview_to_df(page_name_list, start_date, stop_date)

In [90]:
df.to_excel('raw_data//wiki.xlsx',index=False)

## 3) Сбор данных по просмотрам Yandex

In [32]:
import argparse
import datetime
import urllib
from pathlib import Path

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from transliterate import translit
from webdriver_manager.chrome import ChromeDriverManager

YANDEX_WORDSTAT_URL = 'https://wordstat.yandex.com'
YANDEX_WORDSTAT_HISTORY_URL = f'{YANDEX_WORDSTAT_URL}/#!/history?period=weekly&regions=225&words='
OUTPUT_DATA_FOLDER = 'downloaded_data'
SHORT_TIMEOUT = 5
LONG_TIMEOUT = 180
HEADER_COLUMN_NAMES = ['search_query', 'period_start', 'period_end', 'absolute_value']
DATE_FORMAT = "%Y-%m-%d_%H-%M-%S"

LOGIN_XPATH = "//td[@class='b-head-userinfo__entry']/a/span"
USERNAME_XPATH = "//div[@class='b-domik__username']/span/span/input"
PASSWORD_XPATH = "//div[@class='b-domik__password']/span/span/input"
SUBMIT_XPATH = "//div[@class='b-domik__button']/span"
SEARCH_RESULTS_TABLE_XPATH = "//div[@class='b-history__table-box']"
CAPTCHA_INPUT_XPATH = "//td[@class='b-page__captcha-input-td']/span/span/input"
SEARCH_INPUT_XPATH = "//td[@class='b-search__col b-search__input']/span"

request_words = ['потеря обоняния', 'симптомы короновируса', 'Анализы на короновирус', 'Признаки короновируса',
                 'КТ легких', 'Регламент лечения короновирусной инфекции', 'Схемы лечения препаратами короновируса',
                 'Показания для госпитализации с подозрением на коронавирусную болезнь',
                 'Купить противовирусные препараты (Фавипиравир, Ремдесивир)', 'Купить Гидроксихлорохин',
                 'Купить амоксициллин, азитромицин, левофлоксацин, моксифлоксацин', 'Купить антикоагулянты для лечения',
                 'Как защититься от короновируса', 'КОВИД-19', 'потеря вкуса', 'кашель', 'поражение сосудистой стенки',
                 'оксигенация', 'дыхательная функция', 'сатурация', 'пандемия', 'респираторная вирусная инфекция',
                 'SARS-Cov-2', 'пневмония', 'дыхательная недостаточность', 'насморк', 'мышечная слабость',
                 'высокая температура', 'озноб', 'рисунок матовое стекло', 'одышка', 'бессонница',
                 'положительный результат теста на ПЦР', 'наличие антител М', 'наличие антител G', 'Коронавир',
                 'Парацетомол', 'Дексаметазон', 'Уровень кислорода в крови', 'Кислород в крови', 'Вакцинация',
                 'Спутник V', 'иммунитет', 'АстраЗенека', 'Pfizer', 'побочные эффекты', 'осложенния',
                 'противопоказания', 'тромбоз', 'ЭпиВакКорона', 'КовиВак', 'инкубационный период', 'вирулентность',
                 'мутации', 'Британский штамм', 'маска', 'перчатки', 'срок жизни вируса на поверхности', 'профилактика',
                 'антитела g короновирус ковид -19', 'антитела к ковиду показатели', 'вакцина гам ковид вак',
                 'вакцина гам ковид вак и спутник м это одно и тоже или нет', 'вакцины от ковида',
                 'гам ковид вак или спутник м', 'гам-ковид-вак', 'гам-ковид-вак и спутник одно и тоже', 'ковид',
                 'ковид статистика россия', 'прививка от ковид -19 спутник инструкция противопоказания',
                 'прививки от ковида', 'регистр ковид']


def site_login(driver, login, password):
    driver.implicitly_wait(1)
    driver.get(YANDEX_WORDSTAT_URL)
    login_element = driver.find_element(By.XPATH, LOGIN_XPATH)
    login_element.click()

    username_input = driver.find_element(By.XPATH, USERNAME_XPATH)
    password_input = driver.find_element(By.XPATH, PASSWORD_XPATH)
    submit_span = driver.find_element(By.XPATH, SUBMIT_XPATH)

    username_input.send_keys(login)
    password_input.send_keys(password)

    submit_span.click()


def is_captcha_visible(driver):
    try:
        captcha_input = WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.XPATH, CAPTCHA_INPUT_XPATH)))
        return captcha_input.is_displayed()
    except TimeoutException:
        return False


def solve_captcha(driver):
    driver.switch_to.window(driver.current_window_handle)

    try:
        WebDriverWait(driver, SHORT_TIMEOUT).until(EC.visibility_of_element_located((By.XPATH, CAPTCHA_INPUT_XPATH)))
        WebDriverWait(driver, LONG_TIMEOUT).until_not(EC.visibility_of_element_located((By.XPATH, CAPTCHA_INPUT_XPATH)))
    except TimeoutException:
        pass


def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find_all('tr', {"class": ["odd", "even"]})
    rows = []
    for tr in trs:
        td_iterator = tr.findChildren("td", recursive=False)
        f1, f2 = td_iterator[0].text.replace(u'\xa0', ' ').replace(' ', '').split('-')
        f3 = td_iterator[2].text
        rows.append((f1, f2, f3))

    return rows


def check_for_captcha_and_solve_it(driver):
    search_span = driver.find_element(By.XPATH, SEARCH_INPUT_XPATH)

    try:
        search_span.click()
    except ElementClickInterceptedException:
        while is_captcha_visible(driver):
            solve_captcha(driver)

    search_span_clickable = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, SEARCH_INPUT_XPATH)))
    search_span_clickable.click()


def write_csv_file(search_query_words, data_rows, path, divisor='|'):
    f_name = translit(search_query_words, 'ru', reversed=True).replace(' ', '_')
    file_full_path = f"{path}/{f_name}.csv"
    with open(file_full_path, 'w') as f:
        f.write(f"{divisor.join(HEADER_COLUMN_NAMES)}\n")
        for row in data_rows:
            f.write(f"{divisor.join([search_query_words, row[0], row[1], row[2]])}\n")


def try_and_parse_data(driver):
    try:
        stats_table = driver.find_element(By.XPATH, SEARCH_RESULTS_TABLE_XPATH)
        stats_table_visible = WebDriverWait(driver, 10).until(
            EC.visibility_of(stats_table)
        )

        return stats_table_visible.get_attribute('innerHTML')
    except NoSuchElementException:
        # nothing found
        return None


def parse_and_write_to_file(search_query_words, table_html, path):
    data_rows = parse_html(table_html)
    write_csv_file(search_query_words, data_rows, path)


def parse_content_by_url(driver, search_query_words, path):
    encoded_words = urllib.parse.quote(search_query_words.encode('utf-8'))
    url = YANDEX_WORDSTAT_HISTORY_URL + encoded_words
    driver.get(url)

    check_for_captcha_and_solve_it(driver)

    data = try_and_parse_data(driver)
    if data:
        parse_and_write_to_file(search_query_words, data, path)


def create_download_folder_if_not_exists(path, create_subfolder):
    Path(path).mkdir(parents=True, exist_ok=True)
    if create_subfolder:
        t = datetime.datetime.now()
        subfolder_name = t.strftime(DATE_FORMAT)
        final_path = f"{path}/{subfolder_name}"
        Path(final_path).mkdir()

        return final_path


def parse_arguments():
    parser = argparse.ArgumentParser(description='Parse yandex wordstat for query results')
    parser.add_argument('username', type=str, nargs=1,
                        help='username to authenticate with yandex')
    parser.add_argument('password', type=str, nargs=1,
                        help='password to authenticate with yandex')

    args = parser.parse_args()
    return args.username[0], args.password[0]


def main():
    yandex_login, yandex_password = parse_arguments()
    output_data_path = create_download_folder_if_not_exists(OUTPUT_DATA_FOLDER, True)
    with webdriver.Chrome(ChromeDriverManager().install()) as driver:
        site_login(driver, yandex_login, yandex_password)
        for word in request_words:
            parse_content_by_url(driver, word, output_data_path)


if __name__ == '__main__':
    main()


1

Данные по яндексу есть, но обновление этих данных находиться в процессе доработки

## 4) Сбор официальной статистики по заболевшим в Covid-2019 в РФ

In [69]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

In [70]:
l = data.keys().tolist()
l.remove('new_cases')
l.remove('date')
l.remove('location')
data.drop(columns = l, inplace=True)
data_Russia = data[data['location'] == 'Russia']

In [76]:
data_Russia['w'] = pd.to_datetime(data_Russia['date'], format = '%Y-%m-%d').dt.strftime("%V")
data_Russia['y'] = pd.to_datetime(data_Russia['date'], format = '%Y-%m-%d').dt.strftime("%Y")
df = data_Russia.groupby(['w','y'])['new_cases'].sum()
DF_mdate = data_Russia.groupby(['w','y'])['date'].max()
df = pd.DataFrame(df)
DF_mdate = pd.DataFrame(DF_mdate)
D_col = DF_mdate['date']
df.insert(0, 'date', D_col)
df.sort_values(by = ['y', 'w'], inplace=True)
target = df.reset_index()
target.drop(['w','y'], axis = 1, inplace=True)
New_cases_Covid_Russia = target.T
target_c= New_cases_Covid_Russia#[range(12,64)]
target_t = target_c.T
target_t = target_t.reset_index(drop = True)
target_c = target_t.T
target_f = target_c.T
target_f.to_excel('raw_data//target_.xlsx',index=False)
#финальный таргет
target_f

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,new_cases
0,2020-02-02,2
1,2020-02-09,0
2,2020-02-16,0
3,2020-02-23,0
4,2020-03-01,0
...,...,...
64,2021-04-25,59596
65,2021-05-02,59836
66,2021-05-09,56145
67,2021-05-13,32682
